# Simple CNN text classification
Inspired by https://towardsdatascience.com/text-classification-with-cnns-in-pytorch-1113df31e79f

## Load, format data
Should be fed in as a sentence of word token IDs

In [1]:
# Load in data
import pandas as pd
data = pd.read_csv('/data/tumblr_community_identity/dataset114k/matched_reblogs_nonreblogs_dataset114k.csv')
data.columns

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (12,29,38,42,51,54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['post_id_reblog', 'activity_time_epoch_reblog', 'activity_date_reblog',
       'post_title_reblog', 'post_short_url_reblog', 'post_slug_reblog',
       'post_type_reblog', 'post_caption_reblog', 'post_format_reblog',
       'post_note_count_reblog', 'reblogged_from_post_id_reblog',
       'created_time_epoch_reblog', 'updated_time_epoch_reblog',
       'source_url_reblog', 'post_id_nonreblog',
       'activity_time_epoch_nonreblog', 'activity_date_nonreblog',
       'post_title_nonreblog', 'post_short_url_nonreblog',
       'post_slug_nonreblog', 'post_type_nonreblog', 'post_caption_nonreblog',
       'post_format_nonreblog', 'post_note_count_nonreblog',
       'reblogged_from_post_id_nonreblog', 'created_time_epoch_nonreblog',
       'updated_time_epoch_nonreblog', 'source_url_nonreblog',
       'tumblog_id_follower_reblog', 'activity_time_epoch_follower_reblog',
       'tumblr_blog_name_follower_reblog', 'tumblr_blog_title_follower_reblog',
       'tumblr_blog_description_foll

In [3]:
# Build vocabulary, convert to word indices
from collections import defaultdict

w2i = defaultdict(lambda: len(w2i))

selected = data.loc[:, ['processed_tumblr_blog_description_follower_reblog', 'processed_tumblr_blog_description_followee_reblog',
                    'processed_tumblr_blog_description_followee_nonreblog']]
selected['desc_follower'] = selected['processed_tumblr_blog_description_follower_reblog'].astype(str)
selected['desc_followee_reblog'] = selected['processed_tumblr_blog_description_followee_reblog'].astype(str)
selected['desc_followee_nonreblog'] = selected['processed_tumblr_blog_description_followee_nonreblog'].astype(str)
selected

,processed_tumblr_blog_description_follower_reblog,processed_tumblr_blog_description_followee_reblog,processed_tumblr_blog_description_followee_nonreblog,desc_follower,desc_followee_reblog,desc_followee_nonreblog
0,the icon/profile pic is a throwback selfie of ...,23|gay|geek|mild kink|europe my stuff validati...,"justin / 28 / phd student / cambridge , uk / a...",the icon/profile pic is a throwback selfie of ...,23|gay|geek|mild kink|europe my stuff validati...,"justin / 28 / phd student / cambridge , uk / a..."
1,do n't sweat the small stuff,promote your tumblr ! | devil-utopia | | | twi...,creative director and artist based out of nyc ...,do n't sweat the small stuff,promote your tumblr ! | devil-utopia | | | twi...,creative director and artist based out of nyc ...
2,"cute but ... //do n't worry , get a tattoo.//",kübra💫i̇stanbul/18,collection of corvette pictures and videos,"cute but ... //do n't worry , get a tattoo.//",kübra💫i̇stanbul/18,collection of corvette pictures and videos
3,queer|oh|27|they/them,i refuse to be a cliché .,"hey baby , when i write , i 'm the hero of my ...",queer|oh|27|they/them,i refuse to be a cliché .,"hey baby , when i write , i 'm the hero of my ..."
4,i ’ m just a queer nerd who likes to build thi...,obliviate ...,jess . 26 . australian . not spoiler or porn f...,i ’ m just a queer nerd who likes to build thi...,obliviate ...,jess . 26 . australian . not spoiler or porn f...
...,...,...,...,...,...,...
110917,q // nineteen // wrapped around your finger de...,★ we need to talk ★,génèvieve // 19 // i would give my last dying ...,q // nineteen // wrapped around your finger de...,★ we need to talk ★,génèvieve // 19 // i would give my last dying ...
110918,"sensibile , pazzamente innamorata di te",instagram : _tought.face_,sw:57kg | cw : 58kg | gw:45kg hi im fatty bitch,"sensibile , pazzamente innamorata di te",instagram : _tought.face_,sw:57kg | cw : 58kg | gw:45kg hi im fatty bitch
110919,fake it till you make it ..,be better and do better “ +966 +965 “,soundcloud,fake it till you make it ..,be better and do better “ +966 +965 “,soundcloud
110920,终点回到原点享受那走不完的路,约炮平台 -- -- 男生+微信xiehou-8女生+微信nvsheng-2018入群约炮,为成功找方法，不为失败找借口,终点回到原点享受那走不完的路,约炮平台 -- -- 男生+微信xiehou-8女生+微信nvsheng-2018入群约炮,为成功找方法，不为失败找借口


In [4]:
words = selected[['desc_follower', 'desc_followee_reblog', 'desc_followee_nonreblog']].agg(' '.join, axis=1)
words

0         the icon/profile pic is a throwback selfie of ...
1         do n't sweat the small stuff promote your tumb...
2         cute but ... //do n't worry , get a tattoo.// ...
3         queer|oh|27|they/them i refuse to be a cliché ...
4         i ’ m just a queer nerd who likes to build thi...
                                ...                        
110917    q // nineteen // wrapped around your finger de...
110918    sensibile , pazzamente innamorata di te instag...
110919    fake it till you make it .. be better and do b...
110920    终点回到原点享受那走不完的路 约炮平台 -- -- 男生+微信xiehou-8女生+微信nv...
110921    stella /18/ heraklion ig : stella_chatz new le...
Length: 110922, dtype: object

In [6]:
from collections import Counter
words_ctr = Counter([w for s in words.tolist() for w in s.split()])
words_ctr.most_common(20)

[('.', 429189),
 (',', 380974),
 ('i', 208443),
 ('and', 174622),
 ('a', 115807),
 ('to', 112716),
 ('the', 110937),
 ('|', 107290),
 ('my', 94493),
 ('!', 93348),
 ('of', 89994),
 (':', 83884),
 ('you', 81613),
 ('me', 72282),
 ('is', 70665),
 ('blog', 51912),
 ('in', 51357),
 (')', 45709),
 ('for', 45486),
 ("'m", 43160)]

In [17]:
vocab = dict()
for i, (w, count) in enumerate(words_ctr.most_common(100000)):
    vocab[w] = i+1
len(vocab)

100000

In [18]:
for user in ['follower', 'followee_reblog', 'followee_nonreblog']:
    print(user)
    selected[f'indices_{user}'] = selected[f'desc_{user}'].map(lambda x: [vocab[w] for w in x.split() if w in vocab])
selected[[col for col in selected.columns if 'indices' in col]]

follower
followee_reblog
followee_nonreblog


,indices_follower,indices_followee_reblog,indices_followee_nonreblog
0,"[7, 521, 15, 5, 22363, 3049, 11, 14, 319, 2392...","[9, 75, 9688, 389, 22, 15, 33, 5, 195, 16, 2, ...","[3834, 41, 555, 41, 3835, 256, 41, 12694, 2, 3..."
1,"[30, 43, 5532, 7, 705, 75]","[1271, 53, 121, 10, 8, 8, 8, 8, 122, 10, 8, 11...","[1168, 3640, 4, 193, 833, 100, 11, 655, 1, 3, ..."
2,"[354, 45, 49, 40825, 43, 2457, 2, 120, 5]",[],"[472, 11, 145, 4, 310]"
3,[],"[3, 5427, 6, 29, 5, 14734, 1]","[207, 559, 2, 179, 3, 280, 2, 3, 20, 7, 1253, ..."
4,"[3, 50, 110, 38, 5, 286, 344, 73, 358, 6, 3125...",[49],"[1487, 1, 317, 1, 1327, 1, 33, 856, 32, 195, 8..."
...,...,...,...
110917,"[2466, 70, 2579, 70, 7349, 396, 53, 4934, 3982...","[548, 83, 214, 6, 191, 548]","[71490, 70, 108, 70, 3, 188, 397, 9, 745, 2270..."
110918,"[12609, 2, 85565, 18033, 247, 654]","[114, 12]","[49899, 8, 748, 12, 20482, 8, 38563, 103, 138,..."
110919,"[1650, 25, 2055, 13, 127, 25, 174]","[29, 386, 4, 30, 386, 277, 8103, 6590, 277]",[2084]
110920,[],"[21624, 65, 65, 24684]",[56073]


In [25]:
# Pad to length
desc_length = 30

def pad(inds):
    while len(inds) < desc_length:
        inds.insert(len(inds), 0)
    return inds[:desc_length]

for user in ['follower', 'followee_reblog', 'followee_nonreblog']:
    selected[f'indices_{user}'] = selected[f'indices_{user}'].map(lambda x: pad(x))
selected[[col for col in selected.columns if 'indices' in col]]

,indices_follower,indices_followee_reblog,indices_followee_nonreblog
0,"[7, 521, 15, 5, 22363, 3049, 11, 14, 319, 2392...","[9, 75, 9688, 389, 22, 15, 33, 5, 195, 16, 2, ...","[3834, 41, 555, 41, 3835, 256, 41, 12694, 2, 3..."
1,"[30, 43, 5532, 7, 705, 75, 0, 0, 0, 0, 0, 0, 0...","[1271, 53, 121, 10, 8, 8, 8, 8, 122, 10, 8, 11...","[1168, 3640, 4, 193, 833, 100, 11, 655, 1, 3, ..."
2,"[354, 45, 49, 40825, 43, 2457, 2, 120, 5, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[472, 11, 145, 4, 310, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 5427, 6, 29, 5, 14734, 1, 0, 0, 0, 0, 0, 0...","[207, 559, 2, 179, 3, 280, 2, 3, 20, 7, 1253, ..."
4,"[3, 50, 110, 38, 5, 286, 344, 73, 358, 6, 3125...","[49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1487, 1, 317, 1, 1327, 1, 33, 856, 32, 195, 8..."
...,...,...,...
110917,"[2466, 70, 2579, 70, 7349, 396, 53, 4934, 3982...","[548, 83, 214, 6, 191, 548, 0, 0, 0, 0, 0, 0, ...","[71490, 70, 108, 70, 3, 188, 397, 9, 745, 2270..."
110918,"[12609, 2, 85565, 18033, 247, 654, 0, 0, 0, 0,...","[114, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[49899, 8, 748, 12, 20482, 8, 38563, 103, 138,..."
110919,"[1650, 25, 2055, 13, 127, 25, 174, 0, 0, 0, 0,...","[29, 386, 4, 30, 386, 277, 8103, 6590, 277, 0,...","[2084, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
110920,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[21624, 65, 65, 24684, 0, 0, 0, 0, 0, 0, 0, 0,...","[56073, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [27]:
# Feed random labels
import numpy as np

def add_random_labels(data):
    """ Add random 0 and 1 labels for ordering reblog/nonreblogs
        for learning-to-rank organization """
    half_len = int(len(data)/2)
    np.random.seed(9)
    labels = [0]*half_len + [1]*half_len
    np.random.shuffle(labels)
    data['label'] = labels
    return data

data = add_random_labels(selected)
data

,processed_tumblr_blog_description_follower_reblog,processed_tumblr_blog_description_followee_reblog,processed_tumblr_blog_description_followee_nonreblog,desc_follower,desc_followee_reblog,desc_followee_nonreblog,indices_follower,indices_followee_reblog,indices_followee_nonreblog,label
0,the icon/profile pic is a throwback selfie of ...,23|gay|geek|mild kink|europe my stuff validati...,"justin / 28 / phd student / cambridge , uk / a...",the icon/profile pic is a throwback selfie of ...,23|gay|geek|mild kink|europe my stuff validati...,"justin / 28 / phd student / cambridge , uk / a...","[7, 521, 15, 5, 22363, 3049, 11, 14, 319, 2392...","[9, 75, 9688, 389, 22, 15, 33, 5, 195, 16, 2, ...","[3834, 41, 555, 41, 3835, 256, 41, 12694, 2, 3...",0
1,do n't sweat the small stuff,promote your tumblr ! | devil-utopia | | | twi...,creative director and artist based out of nyc ...,do n't sweat the small stuff,promote your tumblr ! | devil-utopia | | | twi...,creative director and artist based out of nyc ...,"[30, 43, 5532, 7, 705, 75, 0, 0, 0, 0, 0, 0, 0...","[1271, 53, 121, 10, 8, 8, 8, 8, 122, 10, 8, 11...","[1168, 3640, 4, 193, 833, 100, 11, 655, 1, 3, ...",1
2,"cute but ... //do n't worry , get a tattoo.//",kübra💫i̇stanbul/18,collection of corvette pictures and videos,"cute but ... //do n't worry , get a tattoo.//",kübra💫i̇stanbul/18,collection of corvette pictures and videos,"[354, 45, 49, 40825, 43, 2457, 2, 120, 5, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[472, 11, 145, 4, 310, 0, 0, 0, 0, 0, 0, 0, 0,...",0
3,queer|oh|27|they/them,i refuse to be a cliché .,"hey baby , when i write , i 'm the hero of my ...",queer|oh|27|they/them,i refuse to be a cliché .,"hey baby , when i write , i 'm the hero of my ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 5427, 6, 29, 5, 14734, 1, 0, 0, 0, 0, 0, 0...","[207, 559, 2, 179, 3, 280, 2, 3, 20, 7, 1253, ...",1
4,i ’ m just a queer nerd who likes to build thi...,obliviate ...,jess . 26 . australian . not spoiler or porn f...,i ’ m just a queer nerd who likes to build thi...,obliviate ...,jess . 26 . australian . not spoiler or porn f...,"[3, 50, 110, 38, 5, 286, 344, 73, 358, 6, 3125...","[49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1487, 1, 317, 1, 1327, 1, 33, 856, 32, 195, 8...",1
...,...,...,...,...,...,...,...,...,...,...
110917,q // nineteen // wrapped around your finger de...,★ we need to talk ★,génèvieve // 19 // i would give my last dying ...,q // nineteen // wrapped around your finger de...,★ we need to talk ★,génèvieve // 19 // i would give my last dying ...,"[2466, 70, 2579, 70, 7349, 396, 53, 4934, 3982...","[548, 83, 214, 6, 191, 548, 0, 0, 0, 0, 0, 0, ...","[71490, 70, 108, 70, 3, 188, 397, 9, 745, 2270...",0
110918,"sensibile , pazzamente innamorata di te",instagram : _tought.face_,sw:57kg | cw : 58kg | gw:45kg hi im fatty bitch,"sensibile , pazzamente innamorata di te",instagram : _tought.face_,sw:57kg | cw : 58kg | gw:45kg hi im fatty bitch,"[12609, 2, 85565, 18033, 247, 654, 0, 0, 0, 0,...","[114, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[49899, 8, 748, 12, 20482, 8, 38563, 103, 138,...",1
110919,fake it till you make it ..,be better and do better “ +966 +965 “,soundcloud,fake it till you make it ..,be better and do better “ +966 +965 “,soundcloud,"[1650, 25, 2055, 13, 127, 25, 174, 0, 0, 0, 0,...","[29, 386, 4, 30, 386, 277, 8103, 6590, 277, 0,...","[2084, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
110920,终点回到原点享受那走不完的路,约炮平台 -- -- 男生+微信xiehou-8女生+微信nvsheng-2018入群约炮,为成功找方法，不为失败找借口,终点回到原点享受那走不完的路,约炮平台 -- -- 男生+微信xiehou-8女生+微信nvsheng-2018入群约炮,为成功找方法，不为失败找借口,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[21624, 65, 65, 24684, 0, 0, 0, 0, 0, 0, 0, 0,...","[56073, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [28]:
# Arrange into proper order
def rank_feature_transform(desc_followers, desc_followee_reblogs, desc_followee_nonreblogs, labels):
    """ Transform features to come up with comparison features for
        learning-to-rank formulation """
    comparison_feats = []
    for desc_follower, desc_followee_reblog, desc_followee_nonreblog, label in zip(
        desc_followers, desc_followee_reblogs, desc_followee_nonreblogs, labels):
        if label == 0:
            comparison_feats.append(desc_follower + desc_followee_reblog + desc_followee_nonreblog)
        else:
            comparison_feats.append(desc_follower + desc_followee_nonreblog + desc_followee_reblog)
    return np.array(comparison_feats)

ordered = rank_feature_transform(data['indices_follower'], data['indices_followee_reblog'], 
                                 data['indices_followee_nonreblog'], data['label'])
ordered.shape

(110922, 90)

## Build, train model

In [29]:
from dataclasses import dataclass

@dataclass
class Parameters:
    # Preprocessing parameeters
    seq_len: int = 90
    num_words: int = 100000
    
    # Model parameters
    embedding_size: int = 64
    out_size: int = 32
    stride: int = 2
    
    # Training parameters
    epochs: int = 10
    batch_size: int = 12
    learning_rate: float = 0.001

In [35]:
model = TextClassifier(Parameters)

In [36]:
from torch.utils.data import Dataset, DataLoader

class DatasetMapper(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
      
    def __len__(self):
        return len(self.x)
       
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [39]:
import torch.optim as optim

# Initialize dataset maper
params = Parameters()
n_train = 10000
X_train = ordered[:n_train]
y_train = data['label'].values[:n_train]
train = DatasetMapper(X_train, y_train)
# test = DatasetMapper(data['x_test'], data['y_test'])

# Initialize loaders
loader_train = DataLoader(train, batch_size=params.batch_size)
# loader_test = DataLoader(test, batch_size=params.batch_size)

# Define optimizer
optimizer = optim.RMSprop(model.parameters(), lr=params.learning_rate)

In [41]:
import torch

# Starts training phase
for epoch in range(params.epochs):
    print(epoch)
    # Set model in training model
    model.train()
    predictions = []
    # Starts batch training
    for x_batch, y_batch in loader_train:

        y_batch = y_batch.type(torch.FloatTensor)

        # Feed the model
        y_pred = model(x_batch)

        # Loss calculation
        loss = F.binary_cross_entropy(y_pred, y_batch)

        # Clean gradientes
        optimizer.zero_grad()

        # Gradients calculation
        loss.backward()

        # Gradients update
        optimizer.step()

        # Save predictions
        predictions += list(y_pred.detach().numpy())

0
1
2
3


KeyboardInterrupt: 

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class TextClassifier(nn.ModuleList):
    def __init__(self, params):
        super(TextClassifier, self).__init__()
  
        # Parameters regarding text preprocessing
        self.seq_len = params.seq_len
        self.num_words = params.num_words
        self.embedding_size = params.embedding_size
        
        # Dropout definition
        self.dropout = nn.Dropout(0.25)
        
        # CNN parameters definition
        # Kernel sizes
        self.kernel_1 = 2
        self.kernel_2 = 3
        self.kernel_3 = 4
        self.kernel_4 = 5
        
        # Output size for each convolution
        self.out_size = params.out_size
        # Number of strides for each convolution
        self.stride = params.stride
        
        # Embedding layer definition
        self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
        
        # Convolution layers definition
        self.conv_1 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_1, self.stride)
        self.conv_2 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_2, self.stride)
        self.conv_3 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_3, self.stride)
        self.conv_4 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_4, self.stride)
        
        # Max pooling layers definition
        self.pool_1 = nn.MaxPool1d(self.kernel_1, self.stride)
        self.pool_2 = nn.MaxPool1d(self.kernel_2, self.stride)
        self.pool_3 = nn.MaxPool1d(self.kernel_3, self.stride)
        self.pool_4 = nn.MaxPool1d(self.kernel_4, self.stride)
        
        # Fully connected layer definition
        self.fc = nn.Linear(self.in_features_fc(), 1)

    def in_features_fc(self):
        '''Calculates the number of output features after Convolution + Max pooling

        Convolved_Features = ((embedding_size + (2 * padding) - dilation * (kernel - 1) - 1) / stride) + 1
        Pooled_Features = ((embedding_size + (2 * padding) - dilation * (kernel - 1) - 1) / stride) + 1

        source: https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
        '''
        # Calculate size of convolved/pooled features for convolution_1/max_pooling_1 features
        out_conv_1 = ((self.embedding_size - 1 * (self.kernel_1 - 1) - 1) / self.stride) + 1
        out_conv_1 = math.floor(out_conv_1)
        out_pool_1 = ((out_conv_1 - 1 * (self.kernel_1 - 1) - 1) / self.stride) + 1
        out_pool_1 = math.floor(out_pool_1)

        # Calculate size of convolved/pooled features for convolution_2/max_pooling_2 features
        out_conv_2 = ((self.embedding_size - 1 * (self.kernel_2 - 1) - 1) / self.stride) + 1
        out_conv_2 = math.floor(out_conv_2)
        out_pool_2 = ((out_conv_2 - 1 * (self.kernel_2 - 1) - 1) / self.stride) + 1
        out_pool_2 = math.floor(out_pool_2)

        # Calculate size of convolved/pooled features for convolution_3/max_pooling_3 features
        out_conv_3 = ((self.embedding_size - 1 * (self.kernel_3 - 1) - 1) / self.stride) + 1
        out_conv_3 = math.floor(out_conv_3)
        out_pool_3 = ((out_conv_3 - 1 * (self.kernel_3 - 1) - 1) / self.stride) + 1
        out_pool_3 = math.floor(out_pool_3)

        # Calculate size of convolved/pooled features for convolution_4/max_pooling_4 features
        out_conv_4 = ((self.embedding_size - 1 * (self.kernel_4 - 1) - 1) / self.stride) + 1
        out_conv_4 = math.floor(out_conv_4)
        out_pool_4 = ((out_conv_4 - 1 * (self.kernel_4 - 1) - 1) / self.stride) + 1
        out_pool_4 = math.floor(out_pool_4)

        # Returns "flattened" vector (input for fully connected layer)
        return (out_pool_1 + out_pool_2 + out_pool_3 + out_pool_4) * self.out_size
    
    def forward(self, x):

        # Sequence of tokes is filterd through an embedding layer
        x = self.embedding(x)
        
        # Convolution layer 1 is applied
        x1 = self.conv_1(x)
        x1 = torch.relu(x1)
        x1 = self.pool_1(x1)
        
        # Convolution layer 2 is applied
        x2 = self.conv_2(x)
        x2 = torch.relu((x2))
        x2 = self.pool_2(x2)
        
        # Convolution layer 3 is applied
        x3 = self.conv_3(x)
        x3 = torch.relu(x3)
        x3 = self.pool_3(x3)
        
        # Convolution layer 4 is applied
        x4 = self.conv_4(x)
        x4 = torch.relu(x4)
        x4 = self.pool_4(x4)
        
        # The output of each convolutional layer is concatenated into a unique vector
        union = torch.cat((x1, x2, x3, x4), 2)
        union = union.reshape(union.size(0), -1)
    
        # The "flattened" vector is passed through a fully connected layer
        out = self.fc(union)
        # Dropout is applied		
        out = self.dropout(out)
        # Activation function is applied
        out = torch.sigmoid(out)
        
        return out.squeeze()